In [1]:
# Parameters
msgs = "Ran from Airflow at 2021-11-09T17:11:15.251116+00:00!"


In [2]:
import pandas as pd
import os 
import sqlalchemy
import time

In [3]:
with open("/home/kali/COVID-19-ES/airflow/process/input_config.txt") as f:
    input_file=f.readline()
print("will process:",input_file)

will process: /home/kali/COVID-19-ES/airflow/download_file/downloads/Informe_Comunicacion_20211015.ods


In [4]:
#Here we load the excel file download for the official web site the health ministry
#Remove the 3 last lines Total, vaccins of Armed forces and Health professionals
#Leaving the sheet_name empty refers will take the first sheet in the excel file
dataframe = pd.read_excel(input_file)
dataframe

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Dosis entregadas AstraZeneca (1),Dosis entregadas Janssen (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas con al menos 1 dosis,Nº Personas vacunadas(pauta completada),Nº Personas con dosis adicional,Fecha de la última vacuna registrada (2)
0,Andalucía,9434970,2019500,1528900,333500,13316870,12802416,0.961368,6813703,6662207,47448,2021-10-14 00:00:00
1,Aragón,1531845,275200,265100,64700,2136845,1996629,0.934382,1063909,1043394,16026,2021-10-14 00:00:00
2,Asturias,1224725,162000,220900,43000,1650625,1671298,1.012524,872720,862769,9840,2021-10-14 00:00:00
3,Baleares,1272150,305100,256400,57600,1891250,1597772,0.844823,852811,834662,4430,2021-10-14 00:00:00
4,Canarias,2358690,574700,455800,133650,3522840,3134909,0.889881,1668495,1625164,11550,2021-10-14 00:00:00
5,Cantabria,727305,114900,125200,22200,989605,917527,0.927165,480983,474190,6651,2021-10-14 00:00:00
6,Castilla y Leon,2916525,412400,481200,102200,3912325,3788288,0.968296,1996918,1955431,62850,2021-10-14 00:00:00
7,Castilla La Mancha,2330505,446800,394900,95075,3267280,3015354,0.922894,1613694,1550399,19158,2021-10-14 00:00:00
8,Cataluña,8368460,1604900,1514300,375750,11863410,11086095,0.934478,5937131,5791932,60403,2021-10-14 00:00:00
9,C. Valenciana,5797540,1213650,863900,213450,8088540,7634375,0.943851,4064301,3992155,33993,2021-10-14 00:00:00


In [5]:
cols_names_dict = {'Unnamed: 0': 'community',
             'Total Dosis entregadas (1)': 'Total doses delivered',
             'Dosis administradas (2)': 'Doses administered',
             '% sobre entregadas': '% over delivered',
             'Nº Personas vacunadas(pauta completada)': 'No. People vaccinated',
             'Fecha de la última vacuna registrada (2)': 'Date'}

In [6]:
#rename columns
dataframe.rename(columns=cols_names_dict, inplace=True)
dataframe = dataframe[list(cols_names_dict.values())]
dataframe

,community,Total doses delivered,Doses administered,% over delivered,No. People vaccinated,Date
0,Andalucía,13316870,12802416,0.961368,6662207,2021-10-14 00:00:00
1,Aragón,2136845,1996629,0.934382,1043394,2021-10-14 00:00:00
2,Asturias,1650625,1671298,1.012524,862769,2021-10-14 00:00:00
3,Baleares,1891250,1597772,0.844823,834662,2021-10-14 00:00:00
4,Canarias,3522840,3134909,0.889881,1625164,2021-10-14 00:00:00
5,Cantabria,989605,917527,0.927165,474190,2021-10-14 00:00:00
6,Castilla y Leon,3912325,3788288,0.968296,1955431,2021-10-14 00:00:00
7,Castilla La Mancha,3267280,3015354,0.922894,1550399,2021-10-14 00:00:00
8,Cataluña,11863410,11086095,0.934478,5791932,2021-10-14 00:00:00
9,C. Valenciana,8088540,7634375,0.943851,3992155,2021-10-14 00:00:00


In [7]:
communities = ['Andalucía',
         'Aragón',
         'Asturias ',
         'Baleares',
         'C. Valenciana',
         'Canarias',
         'Cantabria',
         'Castilla La Mancha',
         'Castilla y Leon ',
         'Cataluña',
         'Ceuta',
         'Extremadura',
         'Galicia',
         'La Rioja',
         'Madrid',
         'Melilla',
         'Murcia ',
         'Navarra',
         'País Vasco'
]
dataframe = dataframe.loc[dataframe["community"].isin(communities)]
dataframe

,community,Total doses delivered,Doses administered,% over delivered,No. People vaccinated,Date
0,Andalucía,13316870,12802416,0.961368,6662207,2021-10-14 00:00:00
1,Aragón,2136845,1996629,0.934382,1043394,2021-10-14 00:00:00
2,Asturias,1650625,1671298,1.012524,862769,2021-10-14 00:00:00
3,Baleares,1891250,1597772,0.844823,834662,2021-10-14 00:00:00
4,Canarias,3522840,3134909,0.889881,1625164,2021-10-14 00:00:00
5,Cantabria,989605,917527,0.927165,474190,2021-10-14 00:00:00
6,Castilla y Leon,3912325,3788288,0.968296,1955431,2021-10-14 00:00:00
7,Castilla La Mancha,3267280,3015354,0.922894,1550399,2021-10-14 00:00:00
8,Cataluña,11863410,11086095,0.934478,5791932,2021-10-14 00:00:00
9,C. Valenciana,8088540,7634375,0.943851,3992155,2021-10-14 00:00:00


In [8]:
#Add ccaa_iso columns (iso code of communities)
def format_community(community_name) : 
    communities = {
        "Andalucía": "AN" ,
        "Aragón": "AR",
        "Asturias": "AS",
        "Canarias": "CN",
        "Cantabria": "CB",
        "Castilla y Leon": "CL",
        "Castilla La Mancha": "CM",
        "Cataluña": "CT",
        "Extremadura": "EX",
        "Galicia": "GA",
        "Baleares": "IB",
        "La Rioja": "RI",
        "Madrid": "MD",
        "Murcia": "MC",
        "Navarra": "NC",
        "País Vasco" : "PV",
        "C. Valenciana": "VC",
        "Ceuta": "CE", 
        "Melilla" : "ML"
    }
    return communities[community_name.strip()] if community_name.strip() in list(communities.keys()) else community_name

dataframe["ccaa_iso"] = dataframe["community"].apply(format_community)
#Set index to date
dataframe = dataframe.set_index("Date")
dataframe

/home/kali/.local/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,community,Total doses delivered,Doses administered,% over delivered,No. People vaccinated,ccaa_iso
Date,,,,,,
2021-10-14,Andalucía,13316870,12802416,0.961368,6662207,AN
2021-10-14,Aragón,2136845,1996629,0.934382,1043394,AR
2021-10-14,Asturias,1650625,1671298,1.012524,862769,AS
2021-10-14,Baleares,1891250,1597772,0.844823,834662,IB
2021-10-14,Canarias,3522840,3134909,0.889881,1625164,CN
2021-10-14,Cantabria,989605,917527,0.927165,474190,CB
2021-10-14,Castilla y Leon,3912325,3788288,0.968296,1955431,CL
2021-10-14,Castilla La Mancha,3267280,3015354,0.922894,1550399,CM
2021-10-14,Cataluña,11863410,11086095,0.934478,5791932,CT


In [9]:
def connection(attempts = 3):
    try :
        database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://u5487210@cvp19:jRG2XXi4CwjHvAa@cvp19.mysql.database.azure.com/my_db')
        return database_connection
    except : 
        if attempts>0:
            print("Error!")
            time.sleep(30)
            connection(attempts-1)
    return False


In [10]:
if connection() : 
    dataframe.to_sql(con=connection(), name='vaccins', if_exists='append')
else : 
    print("The error persists")

In [11]:
os.system(f'mv "{input_file}" "/home/kali/COVID-19-ES/airflow/process/processed/"')

0